In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# 1. Descripción del problema a resolver

En este cuadernillo se quiere cuantificar la brecha sanitaria que existe para diversos problemas
de salud relevantes para el Instituto Nacional del Tórax. En específico, se quiere saber si
tales problemas son resueltos por el sistema de salud o no. Para esto, se contrastará la información de 3 fuentes de información:

1. Casos teóricos por problema de salud según incidencia (ET).
2. Casos reales de egresos hospitalarios a nivel país por problema de salud (EP).
3. Casos reales de egresos hospitalarios del INT por problema de salud (EI).

En primer lugar, se calculará la brecha de egresos hospitalarios teóricos y a nivel país. O sea:

$Brecha Pais= ET - EP$

Si $ET > EP$, entonces existe una brecha, y el sistema sanitario ha fallado en resolver el problema de salud. En caso contrario, si se ha resuelto el problema de salud.

En nuestro caso, se quieren identificar los problemas de salud donde $Brecha Pais > 0$, ya que el INT podría hacerse cargo de tales casos sin resolver en el nuevo hospital.

Una vez identificado los problemas de salud donde hay capacidad de crecimiento, se calculará el porcentaje de crecimiento que tendría el INT al asumir el 100% de la brecha del país. Para esto, se utilizará la siguiente fórmula:

$Casos Nuevos del INT = EI + Brecha Pais$

$PorcentajeDeCrecimientoINT = \frac{Casos Nuevos del INT - EI}{EI} * 100$

Esto nos permitirá identificar qué diagnósticos necesitarán un mayor porcentaje de crecimiento que otros.

## Ejemplo de cálculo

A modo de ejemplo, se mostrará un cálculo manual:

$ET = 1055; EP = 493; EI = 220$

Con estos datos, podemos calcular que:

$Brecha Pais = 1055 - 493 = 562$

Por lo tanto, faltan **562** casos que el sistema sanitario falla en atender. Ahora, asumiendo que el INT se hará cargo del 100% de estos casos (sólo para temas de simplicidad. Sin embargo, en ciertos casos es imposible, especialmente tomando en cuenta la cantidad de casos que recibe actualmente el recinto), es posible calcular el % de crecimiento para esta brecha.

$Casos Nuevos del INT = 220 + 562 = 782$

$PorcentajeDeCrecimientoINT = \frac{782 - 220}{220} * 100 = 255\% $

Por lo tanto, el INT debería crecer en un 255% de su capacidad actual para suplir el 100% de la necesidad sanitaria en el diagnóstico X.


# 1.1 Cálculo de Brecha País con Egresos

## Extracción de egresos teóricos (ET)


In [3]:
# Carga la poblacion teorica de los 45 diagnosticos mas relevantes para el INT
poblacion_teorica = pd.read_excel(
    "../data/processed/proyeccion_problemas_de_salud.xlsx", sheet_name="Proyeccion problemas INE"
)

# Solamente deja el codigo CIE de los problemas de salud
poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]

# Extrae los codigos de los diagnosticos relevantes y los deja como indice
DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()
poblacion_teorica = poblacion_teorica.set_index("Diagnostico").sort_index()

# De los casos totales, se asume que una parte (82%) es paciente FONASA. Ademas, se hara una
# correccion para estimar que son pacientes hospitalizados
RATIO_FONASA = 0.82
RATIO_HOSPITALIZADOS = 1
RATIO_AMBULATORIO = 1 - RATIO_HOSPITALIZADOS

In [4]:
# Deja los egresos teoricos a nivel pais
poblacion_teorica_pais = poblacion_teorica.query("Estrato == 'Pais'").copy()

# Selecciona los egresos teoricos hasta 2020
poblacion_teorica_pais = poblacion_teorica_pais[[i for i in range(2017, 2021)]]

# Corrige la poblacion teorica para pacientes FONASA y hospitalizados
poblacion_teorica_pais = round(poblacion_teorica_pais * RATIO_FONASA * RATIO_HOSPITALIZADOS)

## Extracción de egresos a nivel país (EP)


In [5]:
# Lee egresos hospitalarios a nivel país
egresos_pais = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
    usecols=[
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "DIAG1",
        "n_egresos",
        "dias_estada_totales",
        "n_int_q",
        "n_muertos",
    ],
)

# Deja solamente los 45 egresos relevantes para el INT y desde 2017
egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_pais = egresos_pais.query("ANO_EGRESO >= 2017")

## Extracción de egresos INT (EI)


In [6]:
# Obtiene los egresos del Torax desde los egresos totales
egresos_int = egresos_pais.query("ESTABLECIMIENTO_SALUD == 112103").copy()

In [7]:
# Transforma la tabla de egresos del pais a formato de analisis
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()


# Transforma la tabla de egresos int a formato de analisis

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

# tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
# tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

Una vez extraído los datos y haberlos transformados para realizar su análisis, se procederá a calcular las respectivas brechas.

En primer lugar, se obtendrá la brecha a nivel país por problema de salud. Recordemos que la fórmula de cálculo es:

$Brecha Pais= ET - EP$


In [8]:
brecha_pais = (poblacion_teorica_pais - tabla_dinamica_egresos_pais[("n_egresos")]).astype("Int32")
brecha_pais.style.map(visualize.color_negative_red)

,2017,2018,2019,2020
Diagnostico,,,,
C33X,14,22,24,27
C340,879,918,933,998
C341,670,583,588,785
C342,992,993,1029,1053
C343,784,768,802,878
C381,140,144,136,149
C384,-34,-39,-47,-37
C450,-76,-71,-70,-47
C780,669,682,651,775


Los resultados indican que en gran parte de los diagnósticos faltó suplir la demanada del problema
de salud (ej: C340, C341, C342, C343, etc). Los resultados se pueden dividir en distintos tipos
de
problema de salud:

1. **Cáncer de Pulmón (C33X a C782)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando C33X, C384, C450). EL diagnóstico con una mayor necesidad de atención fue
   el C342, con una falta de 1026 egresos.

2. **Patologías Valvulares (I051 a I712)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando I420).


Con estos resultados, es posible identificar todos los diagnósticos donde es necesario cerrar la brecha de atención de salud. Se seleccionaran los diagnósticos donde en al menos 1 de los años entre 2017 y 2020 exista una $Brecha País > 0.$


In [9]:
# Selecciona solamente los diagnosticos donde exista una brecha > 0 en alguno de los anios
brecha_pais_a_resolver = brecha_pais[(brecha_pais > 0).any(axis=1)]
# Obtiene los codigos de los diagnosticos con una brecha
diagnosticos_con_necesidad_sanitaria = brecha_pais_a_resolver.index

In [10]:
print(f"Los diagnósticos que tienen una necesidad sanitaria a nivel país son:\n"
      f"{list(diagnosticos_con_necesidad_sanitaria)}")

Los diagnósticos que tienen una necesidad sanitaria a nivel país son:
['C33X', 'C340', 'C341', 'C342', 'C343', 'C381', 'C780', 'C782', 'D143', 'D381', 'I051', 'I340', 'I350', 'I456', 'I472', 'I495', 'I710', 'I712', 'J47X', 'J679', 'J841', 'J848', 'J849', 'J90X', 'J91X', 'J931', 'M348', 'Q676']


# 1.2 Cálculo de Hospital Basal tomando todos los casos del SSMO

En este apartado, luego de haber identificado los diagnósticos donde existe una brecha de atención,
se estimará la cantidad de casos que deberá atender el INT si es que recibe el 100% de la brecha de pacientes en el SSMO. Dicho en otras palabras, se calculará la necesidad de recursos si es que el INT atiende al 100% de su población en el SSMO.

Para esto, se obtendrá la **población teórica de los diagnósticos donde haya brecha de atención en el país**. Luego, se sumarán estos casos a los egresos INT del 2019. Finalmente, con la cantidad de egresos nuevos, se estimarán los dias de estada necesarios, y la cantidad de camas necesarias para suplir tales días. O sea:

$Población Teórica SSMO (PTSSMO) + Egresos 2019 INT = Casos Nuevos INT$

$Cantidad de Camas Necesarias = \frac{(Casos Nuevos INT * Dias de Estada Promedio Diagnostico)}{365}$

In [11]:
# Los diagnosticos con necesidad sanitaria estan en 
# la variable `diagnoticos_con_necesidad_sanitaria`

# Deja solamente la poblacion teorica del SSMO
poblacion_teorica_ssmo = poblacion_teorica.query("Estrato == 'SSMO'")

# Selecciona los egresos teoricos hasta 2035
poblacion_teorica_ssmo = poblacion_teorica_ssmo[[i for i in range(2017, 2021)]]

# Corrige por la poblacion de FONASA y hospitalizados
poblacion_teorica_ssmo = round(poblacion_teorica_ssmo * RATIO_FONASA * RATIO_HOSPITALIZADOS)

In [12]:
# Aqui se deja un espacio para calcular la brecha a nivel de servicio. Es necesario obtener los
# egresos agrupados a nivel de servicio, para determinar si existe una brecha de atencion a nivel
# de servicio.

# Si se utiliza la brecha nacional, y se obtiene un porcentaje de esta para estimar la brecha del
# servicio, seria una estimacion poco exacta.
HOSPITALES_SSMO = [
    112300,
    112606,
    112609,
    112612,
    112607,
    112608,
    112610,
    112611,
    112613,
    112102,
    112100,
    112101,
    112107,
    114104,
    112104,
    112103,
    112105,
    112106,
]

In [13]:
# Aisla solamente los recintos del SSMO, y calcula los egresos y dias de estada para los diags 
# relevantes

egresos_ssmo = egresos_pais.query("ESTABLECIMIENTO_SALUD.isin(@HOSPITALES_SSMO)")
tabla_dinamica_egresos_ssmo = pd.pivot_table(
    egresos_ssmo,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

In [14]:
# En primer lugar, se calculara la cantidad de dias estada promedio observados y la cantidad de
# camas que se necesitaron para suplir esos dias de estada

# Extrae la cantidad de dias estada promedio del INT
dias_estada_promedio_int = (
    tabla_dinamica_egresos_int["dias_estada_totales"] / tabla_dinamica_egresos_int["n_egresos"]
)

# Calcula la cantidad de camas que necesitó el INT
cantidad_camas_antiguas_int = (tabla_dinamica_egresos_int["dias_estada_totales"] / 365)

In [15]:
print("Resumen de egresos del INT")
display(tabla_dinamica_egresos_int)
print("Cantidad de camas necesitadas INT - 2017 a 2020")
display(round(cantidad_camas_antiguas_int, 4))

Resumen de egresos del INT


dias_estada_totales                   n_egresos               
ANO_EGRESO                2017  2018  2019  2020      2017 2018 2019 2020
DIAG1                                                                    
C33X                         4    22    34    64         1    1    4    7
C340                       290   207   371    51        21   17   24    7
C341                      2360  2379  2767   974       180  220  220   76
C342                       232   386   195    84        19   24   20    7
C343                      1446  1368  1095   427       118  124  120   38
C381                       234   253   223    31        15   21   19    5
C384                       144    62   159    17         9    9   10    3
C450                       232   243   113    79        23   27    8    5
C780                       458   514   450    97        57   63   62   15
C782                       411   615   441   556        29   53   39   51
D143                       465   373   184    49        32   40   28    6
D381                       262   465   713   304        36   85   97   46
E848                       377   602   574   366        26   35   41   23
I051                        99    82    26    75         7    6    3    6
I080                       612   763   493   339        38   42   42   17
I081                       511   981   797   242        29   44   42   14
I340                       815   804   569   454        40   48   37   28
I350                      1964  2084  1999  1126       118  163  148   83
I351                       484   737   680   508        37   52   49   27
I352                       324   446   476   369        31   26   37   21
I420                       396   822   754   310        33   34   30   10
I456                        82    27    63    16        44   25   29   16
I472                       117   119   106   150        22   29   26   24
I495                       173   148   267    70        51   64   91   23
I710                       379   352   483   530        19   24   25   28
I712                       165   139   170   341         8   13   15   29
J398                       124    69   150   729        19   10    9   16
J47X                       643   843   848   294        53   47   42   20
J679                       864   789   969    94        77   70   69   10
J841                      1564  1286  1799   930       108  108  122   58
J848                       278   399   519   275        25   53   52   27
J849                       343   245   233    87        43   23   27   10
J860                       784   793    97   157        22   61   11   10
J869                      1181   520   408   322        39   27   23   18
J90X                       538   685   536   281        54   72   57   27
J91X                         0    10     0     0         0    1    0    0
J931                       417   266   226    70        21   30   19    5
J955                       609   225   451   214        53   35   44   22
J980                        40    73     7    54         4    6    2    7
M348                       290   357   421    63        21   29   27    2
Q211                       170    97   131   122        47   32   38   22
Q231                         1     6    18   263         1    1    2   19
Q676                        46    69    33    24         4   10    5    4
T820                       174   156   348   475        10   17   25   26
T821                       458   201   199   112        40   28   35   18
Z450                       141   243   190    78        71  132  137   57

Cantidad de camas necesitadas INT - 2017 a 2020


ANO_EGRESO,2017,2018,2019,2020
DIAG1,,,,
C33X,0.0110,0.0603,0.0932,0.1753
C340,0.7945,0.5671,1.0164,0.1397
C341,6.4658,6.5178,7.5808,2.6685
C342,0.6356,1.0575,0.5342,0.2301
C343,3.9616,3.7479,3.0000,1.1699
C381,0.6411,0.6932,0.6110,0.0849
C384,0.3945,0.1699,0.4356,0.0466
C450,0.6356,0.6658,0.3096,0.2164
C780,1.2548,1.4082,1.2329,0.2658


In [16]:
print(f"El INT necesito la siguiente cantidad de camas para los 45 diagnosticos mas relevantes:\n"
      f"{cantidad_camas_antiguas_int.sum().round() * 1.2}")

El INT necesito la siguiente cantidad de camas para los 45 diagnosticos mas relevantes:
ANO_EGRESO
2017    70.8
2018    73.2
2019    72.0
2020    40.8
dtype: float64


Una vez calculada la cantidad de camas necesarias para el 2017-2020 con los 45 diagnósticos más relevantes, es necesario calcular la cantidad de camas con los casos nuevos del INT. Para esto se realizará lo siguiente:

1. Se calculará la brecha de atención para los problemas de salud a nivel SSMO. Esto se hará con la siguiente fórmula:

$Brecha SSMO = Población Teórica en SSMO - Egresos Reales en SSMO$

2. De esta brecha, el 100% la asumirá el INT. Con esto es posible calcular los nuevos casos del INT para los años con brecha. En caso de ausencia de brecha, se asumirá que el INT mantendrá la misma cantidad de casos. Por lo tanto:

$Nuevos Casos INT = Brecha SSMO + Casos INT (Sólo si Brecha SSMO > 0) $

$Nuevos Casos INT = Casos INT (Si Brecha SSMO < 0)$

In [17]:
# Calcula la brecha de casos para el SSMO
brecha_ssmo = (poblacion_teorica_ssmo - tabla_dinamica_egresos_ssmo[("n_egresos")]).astype("Int32")
display(brecha_ssmo.style.map(visualize.color_negative_red))

# Ahora, todos los casos donde se ha cumplido la necesidad sanitaria, se reemplazaran por 0
brecha_ssmo[(brecha_ssmo < 0) | (brecha_ssmo.isna())] = 0

,2017,2018,2019,2020
Diagnostico,,,,
C33X,2,2,-1,-5
C340,54,59,57,76
C341,-107,-145,-140,5
C342,56,54,60,76
C343,-44,-46,-42,44
C381,-2,-7,-5,9
C384,-9,-10,-11,-4
C450,-23,-27,-8,-5
C780,-6,5,3,57


In [18]:
# Calcula los nuevos casos del INT si es que asume el 100% de la brecha del SSMO.
nuevos_casos_int_con_brecha_ssmo = tabla_dinamica_egresos_int["n_egresos"] + brecha_ssmo

# Calcula la cantidad de dias de estada estimados con los nuevos casos del INT
dias_estada_estimados_int_con_brecha_ssmo = (
    nuevos_casos_int_con_brecha_ssmo * dias_estada_promedio_int
)

# Finalmente, calcula la cantidad de camas necesarias con los nuevos dias de estada
cantidad_camas_nuevas_int = (dias_estada_estimados_int_con_brecha_ssmo) / 365

In [19]:
print(
    f"El INT necesitaría la siguiente cantidad de camas para los 45 diagnosticos mas relevantes con"
    f" los casos nuevos y antiguos:\n{cantidad_camas_nuevas_int.sum().round()}"
)

El INT necesitaría la siguiente cantidad de camas para los 45 diagnosticos mas relevantes con los casos nuevos y antiguos:
ANO_EGRESO
2017    303.0
2018    435.0
2019    328.0
2020    320.0
dtype: Float64


Ahora, se quiere saber cuáles son los diagnósticos que tuvieron el mayor aumento de camas. Seguramente, los diagnósticos que aumentaron drásticamente sus egresos tendrán el mayor aumento de camas.

In [20]:
round((cantidad_camas_nuevas_int - cantidad_camas_antiguas_int), 2).sum(axis=1).sort_values()

DIAG1
Z450       0.0
I351       0.0
I456       0.0
T821       0.0
I081       0.0
I080       0.0
I712       0.0
E848       0.0
J398       0.0
I352       0.0
J860       0.0
J955       0.0
C450       0.0
C384       0.0
J980       0.0
Q211       0.0
Q231       0.0
T820       0.0
J869       0.0
I420       0.0
Q676      0.11
C33X      0.14
C381      0.15
C341      0.18
I710      0.33
J679      0.33
C780      1.18
M348      1.21
D381      1.25
C343      1.35
J841      3.47
C782      3.62
D143      5.37
J931      5.91
I495      7.93
C340      7.94
C342      8.35
I340      8.81
I051     10.96
J849     19.44
I472     30.78
J848     30.81
J91X    124.41
I350     138.0
J47X    274.27
J90X    484.69
dtype: Float64

---

In [21]:
print("Pais:")
display(tabla_dinamica_egresos_pais)

print("SSMO:")
display(tabla_dinamica_egresos_ssmo)

print("INT:")
display(tabla_dinamica_egresos_int)

Pais:


dias_estada_totales                      n_egresos              \
ANO_EGRESO                2017   2018   2019   2020      2017  2018  2019   
DIAG1                                                                       
C33X                       426    193    155    136        25    18    17   
C340                      1217   1141   1163    661       163   143   148   
C341                      3682   4267   4425   2514       372   478   493   
C342                       494    734    394    478        50    68    52   
C343                      2376   2522   2446   1798       258   293   279   
C381                       474    403    575    319        41    41    52   
C384                       612    449    508    313        39    44    52   
C450                       661    566    560    396        81    76    75   
C780                      3973   3348   3532   2886       373   379   430   
C782                      1691   1839   2130   1835       147   188   211   
D143                      1286   1222    762    537       178   207   160   
D381                      4665   5365   5354   3964       553   641   654   
E848                       891   1188    725    606        58    78    56   
I051                       212    390    422    312        22    39    26   
I080                      1552   1907   1578    865        96   120   110   
I081                       843   1461   1063    457        51    67    60   
I340                      6719   6304   4366   3614       436   413   341   
I350                     16427  18300  17666  12946      1359  1481  1567   
I351                      4155   4062   3698   2788       318   327   313   
I352                      1727   1787   1733   1326        96   126   139   
I420                      5524   5953   6417   3839       489   497   561   
I456                       513    551    504    371       202   190   183   
I472                      3474   3521   3925   2725       429   478   442   
I495                      5499   6143   6091   4860       892  1077  1147   
I710                      5076   6138   6098   5509       429   474   467   
I712                      1371   1519   1305   1582       109   130   122   
J398                      2352    522    829   1721       124    92   106   
J47X                      3941   3747   4168   1938       377   373   395   
J679                      1171   1080   1235    354       106   101   109   
J841                     16577  15087  15943  10135      1666  1633  1697   
J848                      1098   1433   1596   1158       113   172   158   
J849                      4126   5113   5342   2989       502   564   600   
J860                      1535   1436   1041    895        66    95    49   
J869                      9494   7948   8036   5712       417   372   408   
J90X                     24326  25800  27836  20457      2163  2252  2424   
J91X                         0     18      0      0         0     3     0   
J931                      4007   3946   3855   2788       475   481   443   
J955                      1628   2770   1372   1525       167   162   167   
J980                       565    671    625    352       112   105    88   
M348                       432    435    596     93        44    55    45   
Q211                      2208   1918   1565   1374       396   366   409   
Q231                        91    243    217    622        19    29    27   
Q676                       650    745    845    419       149   154   131   
T820                       515    341    700    912        35    41    50   
T821                      1193    881   1134    810       180   167   222   
Z450                      1530   1436   1470    930       718   649   685   

                  
ANO_EGRESO  2020  
DIAG1             
C33X          14  
C340         103  
C341         316  
C342          48  
C343         223  
C381          42  
C384          43  
C450          53  
C780         326  
C782     

SSMO:


dias_estada_totales                   n_egresos               
ANO_EGRESO                2017  2018  2019  2020      2017 2018 2019 2020
DIAG1                                                                    
C33X                         4    22    34    73         1    1    4    8
C340                       297   243   371    51        22   19   24    7
C341                      2433  2428  2784  1000       183  223  221   78
C342                       239   386   201    84        20   24   21    7
C343                      1466  1368  1174   440       120  124  123   39
C381                       234   253   223    31        15   21   19    5
C384                       144    84   175    27         9   10   11    4
C450                       232   243   113    79        23   27    8    5
C780                       619   579   647   155        82   73   78   26
C782                       464   719   670   629        35   63   56   57
D143                       465   373   184    50        32   40   28    7
D381                       302   584   763   357        40   99  105   54
E848                       702   859   688   366        43   47   49   23
I051                        99    82    26    75         7    6    3    6
I080                       628   763   532   339        39   42   43   17
I081                       511   981   864   242        29   44   43   14
I340                       956   807   601   454        46   49   38   28
I350                      2079  2132  2135  1197       126  168  156   90
I351                       484   737   680   573        37   52   49   31
I352                       459   517   521   477        36   30   44   26
I420                       531  1052  1029   369        43   54   54   20
I456                       158    56   113    29        50   29   36   20
I472                       391   367   350   228        43   51   44   37
I495                       529   346   611   278        78   85  117   41
I710                       464   449   593   673        30   39   34   39
I712                       169   149   200   352         9   14   16   31
J398                       624    95   195   740        24   13   21   20
J47X                       718   876   880   305        59   55   47   23
J679                       886   798   998   104        80   72   71   11
J841                      1815  1421  1980  1080       134  128  144   75
J848                       281   471   614   277        26   60   61   28
J849                       423   285   317   146        53   28   33   14
J860                       784   816   173   169        22   63   12   13
J869                      1296   802   615   451        49   40   39   27
J90X                      1164  1484  1137   835       103  134  113   81
J91X                         0    10     0     0         0    1    0    0
J931                       474   382   348   181        30   47   36   19
J955                       792   388   632   618        78   64   89   57
J980                        44    80     7    76         6    8    2   12
M348                       290   357   499    66        21   29   29    3
Q211                       339   245   317   247        95   62   85   52
Q231                         2    10    24   367         2    2    5   23
Q676                       135   144    70   117        21   24   13   21
T820                       180   163   348   500        11   18   25   28
T821                       479   346   252   191        45   45   47   31
Z450                       141   247   193    89        71  134  139   61

INT:


dias_estada_totales                   n_egresos               
ANO_EGRESO                2017  2018  2019  2020      2017 2018 2019 2020
DIAG1                                                                    
C33X                         4    22    34    64         1    1    4    7
C340                       290   207   371    51        21   17   24    7
C341                      2360  2379  2767   974       180  220  220   76
C342                       232   386   195    84        19   24   20    7
C343                      1446  1368  1095   427       118  124  120   38
C381                       234   253   223    31        15   21   19    5
C384                       144    62   159    17         9    9   10    3
C450                       232   243   113    79        23   27    8    5
C780                       458   514   450    97        57   63   62   15
C782                       411   615   441   556        29   53   39   51
D143                       465   373   184    49        32   40   28    6
D381                       262   465   713   304        36   85   97   46
E848                       377   602   574   366        26   35   41   23
I051                        99    82    26    75         7    6    3    6
I080                       612   763   493   339        38   42   42   17
I081                       511   981   797   242        29   44   42   14
I340                       815   804   569   454        40   48   37   28
I350                      1964  2084  1999  1126       118  163  148   83
I351                       484   737   680   508        37   52   49   27
I352                       324   446   476   369        31   26   37   21
I420                       396   822   754   310        33   34   30   10
I456                        82    27    63    16        44   25   29   16
I472                       117   119   106   150        22   29   26   24
I495                       173   148   267    70        51   64   91   23
I710                       379   352   483   530        19   24   25   28
I712                       165   139   170   341         8   13   15   29
J398                       124    69   150   729        19   10    9   16
J47X                       643   843   848   294        53   47   42   20
J679                       864   789   969    94        77   70   69   10
J841                      1564  1286  1799   930       108  108  122   58
J848                       278   399   519   275        25   53   52   27
J849                       343   245   233    87        43   23   27   10
J860                       784   793    97   157        22   61   11   10
J869                      1181   520   408   322        39   27   23   18
J90X                       538   685   536   281        54   72   57   27
J91X                         0    10     0     0         0    1    0    0
J931                       417   266   226    70        21   30   19    5
J955                       609   225   451   214        53   35   44   22
J980                        40    73     7    54         4    6    2    7
M348                       290   357   421    63        21   29   27    2
Q211                       170    97   131   122        47   32   38   22
Q231                         1     6    18   263         1    1    2   19
Q676                        46    69    33    24         4   10    5    4
T820                       174   156   348   475        10   17   25   26
T821                       458   201   199   112        40   28   35   18
Z450                       141   243   190    78        71  132  137   57

# Porcentaje de Casos en Atención Cerrada y Abierta

En este apartado, se quiere calcular el porcentaje de casos resueltos en la atención cerrada y la abierta. Para esto, se tomará como el 100% de los casos a la población teórica calculada según las incidencias. Por otro lado, este dato se contrastará con los egresos nacionales reales para estimar el porcentaje de casos resueltos en la atención cerrada.